In [197]:
import torch

print(torch.__version__)

1.9.1


In [51]:
import torch
import pickle
import pandas as pd
with open('/home/intern/seungjun/testing_jump/dataset/d2_suitability/data/test_data.pickle','rb') as f:
    data = torch.load(f)

df = pd.DataFrame(data)
list(df)


['num_id',
 'id',
 'genre',
 'year',
 'title',
 'text',
 'nudity',
 'violence',
 'profanity',
 'alcohol',
 'frightening',
 'rate']

In [ ]:
df.head()

In [ ]:
print(len(df['text']))

In [55]:
import torch
from torchtext.legacy import data
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.distributions import Categorical
from torchtext.legacy import datasets

import random
import numpy as np
import os.path

try:
    import spacy
except:
    !pip install spacy
    
try:
    spacy.load('en')
except:
    !python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.6 MB 9.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [99]:
print(df['text'][0])

audience cheering thats last years battle year . international bboy competition held france last 15 years united states lost . went concert past weekend overheard high school kids saying bboying longer cool something parents . scared company everything ive built started bboying . obviously still cool france . government subsidizes battle year . government keeping hiphop alive . still cool russia germany korea . cool founded . would explain domestically sales . merchandising record sales ticket sales apparel . first time companys history international sales exceeding domestic . think bboying aint cool long hiphop isnt cool ? mean business company ? hey yo ! get ass . come man . sleep youre dead . see maid doesnt windows ? hell man ? ive calling aint answering . ive busy lately . right well bring busy ass open door . aint prison visit . look like shit . look came sweat came make proposition . gonna let ? okay old thing work ? thats japan versus korea world champions . mean bboying aint b

In [186]:

# GPU 할당 변경하기
GPU_NUM = 3 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

# Additional Infos
if device.type == 'cuda':
    print(torch.cuda.get_device_name(GPU_NUM))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(GPU_NUM)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(GPU_NUM)/1024**3,1), 'GB')

Current cuda device  3
GeForce RTX 3090
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [153]:
TEXT = data.Field(sequential=True, use_vocab = True, tokenize='spacy', lower=True, include_lengths=True)
LABEL = data.LabelField(dtype=torch.float)

datafields = [('text', TEXT), ('label', LABEL)]

result_data=[]
for idx in range(len(df)):
    result_data.append(data.Example.fromlist([df['text'][idx],df['rate'][idx]], datafields))

train_data = data.Dataset(result_data, datafields)

TEXT.build_vocab(train_data, max_size=100000, vectors="glove.6B.100d")
LABEL.build_vocab(train_data)

print(f'Number of training examples: {len(train_data)}')

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")


/home/common/miniconda3/envs/seungjun/lib/python3.7/site-packages/torchtext-0.10.1-py3.7-linux-x86_64.egg/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


Number of training examples: 365
Unique tokens in TEXT vocabulary: 61041
Unique tokens in LABEL vocabulary: 3


In [187]:
train_iterator = data.BucketIterator(train_data,batch_size=50,device='cuda:3',repeat=False)
print(device)


cuda:3


In [116]:
for batch in train_loader:
    break
print(batch.label[0])

tensor(1., device='cuda:2')


In [117]:
label_to_ix = {'negative': 0, 'neutral': 1, 'positive': 2}
print(len(label_to_ix))
print(label_to_ix['negative'])

3
0


In [188]:
N = 5 # Max number of jumps allowed
K = 40 # Max jump size
R = 20 # Number of tokens read between every two jumps

def reward_function(prob, true_label):
  """
  Returns 1 if correct prediction, -1 otherwise
  """
  # print("true_label", "prob", true_label, prob)
  if prob>0.5 and true_label>0.5:
    return torch.tensor(1.0, requires_grad=True)
  if prob<0.5 and true_label<0.5:
    return torch.tensor(1.0, requires_grad=True)
  return torch.tensor(-1.0, requires_grad=True)

class SkipReadingModel(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, gamma=0.99, train_mode=True):
        super().__init__()
        
        # store dimensions and constants
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.gamma = gamma
        self.train_mode = True
        
        # create layers
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm_cell = nn.LSTMCell(input_size = embedding_dim, hidden_size = hidden_dim, bias = True)
        self.jumping_linear = nn.Linear(hidden_dim, K+1)
        self.output_linear = nn.Linear(hidden_dim, output_dim)

        # Baseline weight        
        self.wb = nn.Parameter(data=torch.zeros(self.hidden_dim), requires_grad=True)
        self.cb = nn.Parameter(data=torch.tensor((0.0)), requires_grad=True)
        # Initialize lstm_cell states
        self.initialize_lstm_cell_states()
        
        # Initalize episode number and time number
        self.initialize_for_new_batch()
        self.initialize_time_number()
        
        # Overall reward and loss history
        self.reward_history = []
        self.loss_history = []
        self.training_accuracies = []
        self.validation_accuracies = []
        # torch.tensor((0.0), requires_grad=True) 
    
    def initialize_lstm_cell_states(self):
        self.c = torch.zeros(1, self.hidden_dim, requires_grad=True)
        self.h = torch.zeros(1, self.hidden_dim, requires_grad=True)
            
    def initialize_episode_number(self):
        self.ep = 0
    
    def initialize_time_number(self):
        self.t = 0
    
    def clear_batch_lists(self):
        del self.saved_log_probs[:]
        del self.reward_baselines[:]
        del self.rewards[:]
        del self.label_targets[:]
        del self.label_predictions[:]
        self.initialize_episode_number()
        self.training_accuracy = 0.0
    
    def initialize_for_new_batch(self):
        """
        Cleans history of log probabilities, rewards, targets etc for the last
        batch
        """
        self.initialize_episode_number()
        
        # Episode policy and reward history 
        self.saved_log_probs = [] # log probabilities for each time step t in each episode in batch
        self.rewards = [] # reward at final time step of each episode in batch
        self.reward_baselines = [] # reward baselines for each time step t in each episode in batch

        # Predictions and targets history (for cross entropy loss calculation)
        self.label_predictions = [] # 1 probability for each episode
        self.label_targets = []# 1 label for each episode
        self.training_accuracy = 0.0

        
    def classify(self):
        out = self.output_linear(self.c[0])
        self.label_predictions.append(out)
        if self.train_mode:
          self.rewards.append(reward_function(torch.sigmoid(out), self.label_targets[-1]))
        # return torch.sigmoid(out)
       
    def get_baseline(self):
        return torch.dot(self.wb, self.c[0].detach()) + self.cb
    
    def save_training_accuracy(self):
      correct = 0
      for _r in self.rewards:
        if _r > 0:
          correct += 1
      self.training_accuracy = correct/len(self.rewards)
      self.training_accuracies.append(self.training_accuracy)
      
    
    def forward(self, pack):
        texts, lengths, labels = pack
        embeddeds = self.embedding(texts)
        # embeddeds = nn.utils.rnn.pack_padded_sequence(embeddeds, lengths)
        self.initialize_for_new_batch()

        for episode_number in range(embeddeds.shape[1]):
            
            # load episode data
            self.ep = episode_number
            embedded = embeddeds[:, self.ep, :]
      
            #print(texts.shape, embeddeds.shape, embedded.shape)
            #print(label)
            
            # initialize counters and index
            tokens_read = 0
            jumps_made = 0
            word_index = 0
            words_len = embedded.shape[0]
            self.initialize_lstm_cell_states()
            self.initialize_time_number()
            self.saved_log_probs.append([])
            self.reward_baselines.append([])
            
            if self.train_mode:
              label = labels[self.ep].reshape(1)
              self.label_targets.append(label)
            
            # start iterating through sequence, while skipping some words
            while word_index<words_len and word_index<600:
              self.t += 1
              embedded_word = embedded[word_index]
              #print("embedded_word", embedded_word.shape)
              
              # generate next lstm cell state
              self.h, self.c = self.lstm_cell(torch.reshape(embedded_word, (1, -1)), (self.h, self.c))
              # print('word_index', word_index, 'tokens_read', tokens_read, 'jumps_made', jumps_made)
              
              # if time to jump words
              if tokens_read % R==0:
                
                probs = self.jumping_linear(self.c[0])
                probs = F.softmax(probs)
                categ = Categorical(probs)
                k = categ.sample()
                
                # Add log probability of our chosen action to our history
                self.saved_log_probs[-1].append(categ.log_prob(k))
                self.reward_baselines[-1].append(self.get_baseline())
                
                # classify and move to next episode
                if k==0:
                    self.classify()
                    break
                
                # increment counters
                jumps_made += 1
                word_index+=k
              # if not time to jump, then continue to next word
              else:
                word_index+=1
              
              # increment words read so far counter
              tokens_read += 1
              
              # if max amounts of jump made already, 
              # then classify and move to next episode
              if jumps_made == N:
                self.classify()
                break
            else:
              # print("Finished while loop")
              # raise ValueError('Finshed ')
              self.classify()
        if self.train_mode:
            self.save_training_accuracy()
        return self.label_predictions

In [189]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = 3
seed = 7
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
print(INPUT_DIM)
print(device)
bce = nn.BCEWithLogitsLoss(reduction='mean')
policy_model = SkipReadingModel(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM).to('cuda:3')
print(policy_model)
pretrained_embeddings = TEXT.vocab.vectors
policy_model.embedding.weight.data.copy_(pretrained_embeddings)
if os.path.exists('jump_model.weights'):
  policy_model.load_state_dict(torch.load('jump_model.weights'))
  print("yes")
print(device)
# define the optimiser
optimizer = optim.Adam(policy_model.parameters(), lr=0.001)

61041
cuda:3
SkipReadingModel(
  (embedding): Embedding(61041, 100)
  (lstm_cell): LSTMCell(100, 128)
  (jumping_linear): Linear(in_features=128, out_features=41, bias=True)
  (output_linear): Linear(in_features=128, out_features=3, bias=True)
)
cuda:3


In [180]:
# import torch
# def update_policy():
   
#     #print(len(policy_model.rewards), len(policy_model.label_predictions), len(policy_model.label_targets))
#     #print(len(policy_model.saved_log_probs), len(policy_model.reward_baselines))
    
#     J1 = bce(
#         torch.cat(policy_model.label_predictions),
#         torch.cat(policy_model.label_targets)
#     )
#     J2 = torch.tensor(0.0, device=device, requires_grad=True)
#     J3 = torch.tensor(0.0, device=device, requires_grad=True)
    
#     for R, lps, rbs in zip(policy_model.rewards, policy_model.saved_log_probs, policy_model.reward_baselines):
#         for log_prob, baseline in zip(lps, rbs):
#           weighted_log_prob = (-log_prob * (R - baseline.item()))
#           _baseline_loss = torch.pow(R - baseline, 2)
#           J2 = J2 + weighted_log_prob
#           J3 = J3 + _baseline_loss
    
#     optimizer.zero_grad()
#     policy_loss =  J1 +  (J2/len(policy_model.rewards)) + J3
#     # print('policy_loss', policy_loss)
#     policy_loss.backward(retain_graph=True)
#     optimizer.step()
    
#     policy_model.clear_batch_lists()

In [163]:
# def test_model():
#   policy_model.train_mode = True
#   correct = 0
#   total=0
#   for _data in test_iterator:
#     # get the inputs
#     texts, text_lengths, labels = _data.text[0], _data.text[1], _data.label
#     # print("Input review texts, text_lengths, labels", texts.shape, text_lengths.shape, labels.shape)
#     predictions = policy_model((texts.to(device), text_lengths.to(device), labels.to(device)))
#     for (prediction, label) in zip(predictions, labels):
#       if reward_function(label, prediction) > 0:
#         correct += 1
#       total += 1
#     if total%1000 == 0:
#       print(total)
#     if total%5000 == 0:
#       break
#   print("Test accuracy :", correct/total)
#   policy_model.train_mode = True
#   return correct/total

# def validate_model():
#   policy_model.train_mode = True
#   correct = 0
#   total=0
#   for _data in valid_iterator:
#     # get the inputs
#     texts, text_lengths, labels = _data.text[0], _data.text[1], _data.label
#     # print("Input review texts, text_lengths, labels", texts.shape, text_lengths.shape, labels.shape)
#     predictions = policy_model((texts.to(device), text_lengths.to(device), labels.to(device)))
#     for (prediction, label) in zip(predictions, labels):
#       if reward_function(label, prediction) > 0:
#         correct += 1
#       total += 1
#     if total%1000 == 0:
#       break
#   print("Validation accuracy :", correct/total)
#   policy_model.train_mode = True
#   policy_model.validation_accuracies.append(correct/total)
#   return correct/total


In [196]:
# the epoch loop

with torch.enable_grad():
  # validate_model()
  for epoch in range(10):
      running_reward = 10
      t = 0
      for _data in train_iterator:
          # get the inputs
          texts, text_lengths, labels = _data.text[0], _data.text[1], _data.label
          print("Input review texts, text_lengths, labels", texts.shape, text_lengths.shape, labels.shape)
          print(device)
          prediction = policy_model((texts.to('cuda:3'), text_lengths.to('cuda:3'), labels.to('cuda:3')))
          # print("Prediction", prediction.item())        
          # raise ValueError('Not done')
          t += 1
          if t%2 == 0:
            print("batch no. %d, training accuracy %4.2f" % (t, policy_model.training_accuracy))
          # if t%10 == 0:
            # validate_model()
            #print("wb", policy_model.wb)
            #print("lstm hh hi", policy_model.lstm_cell.weight_hh, policy_model.lstm_cell.weight_ih)
            #print("emb", policy_model.embedding.weight)
            #print("jmp", policy_model.jumping_linear.weight)
            #print("out", policy_model.output_linear.weight)
          if t%1000 == 0:
            break
          # update_policy()
      #running_reward = 0.05 * policy_model.reward_episode + (1 - 0.05) * running_reward
      #print("Epoch %d, reward %4.2f" % (epoch, running_reward))
      print("Epoch %d" % (epoch))
  print('**** Finished Training ****')
# test_model()

Input review texts, text_lengths, labels torch.Size([16989, 50]) torch.Size([50]) torch.Size([50])
cuda:3


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:3 and cuda:2! (when checking arugment for argument index in method wrapper_index_select)